# Libraries

In [87]:
library(plyr)

library(tidyverse)
library(DT) # dependency
library(ggthemes) # dependency

In [88]:
source("tools.R") # Ad-hoc tools

# General variables and functions

In [89]:
se <- function(x) {
    sd(x)/sqrt(length(x))
}

bar_graph <- function(plot_data) {
    plot_data %>%
        ggplot(aes(x=x, y=y))+
        geom_bar(stat="identity", position=position_dodge()) +
        geom_errorbar(aes(ymin=lower_bound, ymax=upper_bound),
                      position=position_dodge(.9),
                      width=0.2 )
}

decimals <- 2 # for rounding

# Import data

In [90]:
directory <- "Data January 2017/"

dat <- get_fc_table(directory)

### Data mutation

In [91]:
# Change column names
colnames(dat) <- c( "row_id",
                    "left_door",
                    "right_door",
                    "rt",
                    "KEY_RESPONSE_TIME",
                    "DISPLAY_TIME", 
                    "key_pressed",
                    "block_level_1_fc",
                    "block",
                    "id",
                    "experiment" )


# Remove ".jpg" of the doors' names; Change {f,j} to {left, right}; Add chosen_door column
dat <- dat %>%
    mutate(left_door = substr(left_door, 0, 7), 
           right_door = substr(right_door, 0, 7),
           key_pressed = ifelse(key_pressed=='f', 'left', 'right'), 
           chosen_door = ifelse(key_pressed=='left', left_door, right_door)) 

Cue values

In [92]:
for(index in 1:length(dat$id)){
    dat$left_speed[index] <- get_cues(dat$experiment[index], dat$left_door[index])[1]
    dat$left_safety[index] <- get_cues(dat$experiment[index], dat$left_door[index])[2]
    
    dat$right_speed[index] <- get_cues(dat$experiment[index], dat$right_door[index])[1]
    dat$right_safety[index] <- get_cues(dat$experiment[index], dat$right_door[index])[2]
    
    dat$chosen_speed[index] <- get_cues(dat$experiment[index], dat$chosen_door[index])[1]
    dat$chosen_safety[index] <- get_cues(dat$experiment[index], dat$chosen_door[index])[2]
}

Add final columns

In [93]:
# Number of cues to check
# ASSUMPTION: when an unknown door is involved, the decision takes 1 step

dat <- dat %>%
    mutate(steps_ttb_speed = ifelse(left_speed == right_speed, 2, 1),
           steps_ttb_safety = ifelse(left_safety == right_safety, 2, 1),
           unknown_door_involved = (left_speed == 0 | right_speed ==0),
           chosen_door_is_unknown = (chosen_speed == 0) ) # Checking one cue is enough

Now that we have the cue values, we can calculate the **predicted** decisions

In [94]:
# Choice per TTB.
# ASSUMPTION: unknown door is never chosen

## TTB Speed

dat[!dat$unknown_door_involved & dat$left_speed > dat$right_speed, 'chosen_ttb_speed'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_speed < dat$right_speed, 'chosen_ttb_speed'] <- 'right'

dat[!dat$unknown_door_involved & dat$left_speed == dat$right_speed & dat$left_safety > dat$right_safety, 'chosen_ttb_speed'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_speed == dat$right_speed & dat$left_safety < dat$right_safety, 'chosen_ttb_speed'] <- 'right'

dat[dat$unknown_door_involved & dat$left_speed == 0 , 'chosen_ttb_speed'] <- 'right'

dat[dat$unknown_door_involved & dat$right_speed == 0 , 'chosen_ttb_speed'] <- 'left'


## TTB Safety

dat[!dat$unknown_door_involved & dat$left_safety > dat$right_safety, 'chosen_ttb_safety'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_safety < dat$right_safety, 'chosen_ttb_safety'] <- 'right'

dat[!dat$unknown_door_involved & dat$left_safety == dat$right_safety & dat$left_speed > dat$right_speed, 'chosen_ttb_safety'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_safety == dat$right_safety & dat$left_speed < dat$right_speed, 'chosen_ttb_safety'] <- 'right'

dat[dat$unknown_door_involved & dat$left_safety == 0 , 'chosen_ttb_safety'] <- 'right'

dat[dat$unknown_door_involved & dat$right_safety == 0 , 'chosen_ttb_safety'] <- 'left'

In [95]:
# When choices are equivalent, the predicted choice is unknown
dat$equivalent_options = FALSE

dat[dat$right_speed == dat$left_speed & dat$right_safety == dat$left_safety, 'equivalent_options'] <- TRUE

dat[dat$equivalent_options == TRUE, 'chosen_ttb_safety'] <- 'unknown'
dat[dat$equivalent_options == TRUE, 'chosen_ttb_speed'] <- 'unknown'

<hr>
<hr>

# Accordance rates

Accordance rate is the proportion of human responses that were correctly predicted

In [96]:
accordance <- function( data, column ) {
    
    column <- enquo(column)
    
    data %>%
        mutate( correctly_predicted = (key_pressed == !!column) ) %>%
        group_by( id ) %>%
        summarize( accordance_rate = mean(correctly_predicted) ) %>%
        summarize( accordance_rate_mean = mean(accordance_rate), 
                   se = se(accordance_rate),
                   lower_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[1],
                   upper_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[2]) %>%
        round( decimals )
}

### 1. Cases where both models predict the same outcome

In [97]:
print('Safety')
dat %>% 
    filter( chosen_ttb_speed == chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( chosen_ttb_speed == chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.92,0.01,0.89,0.95


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.92,0.01,0.89,0.95


### 2. Cases where both models predict different outcomes

In [98]:
print('Safety')
dat %>% 
    filter( chosen_ttb_speed != chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( chosen_ttb_speed != chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.96,0.02,0.92,1


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.04,0.02,0,0.08


### 3. Both choices are equivalent

Rate of 'right' choices. It should be 50%.

In [99]:
dat %>%
    filter( equivalent_options == TRUE ) %>%
    mutate(selected_right = key_pressed == 'right') %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )

selected_right_rate_mean,se,lower_bound,upper_bound
0.55,0.03,0.5,0.6


### 4. Unknown door is involved

In [100]:
print('Safety')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


### 4.2 Unknown door is involved. Digging deeper

Here I check what happens when different types of doors are contrasted against an unknown door

In [101]:
print("Safety, Speed = [1, 1]")
safety_val = 1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [1, -1]")
safety_val = 1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, 1]")
safety_val = -1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, -1]")
safety_val = -1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )

[1] "Safety, Speed = [1, 1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.97,0.01,0.95,0.99


[1] "Safety, Speed = [1, -1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.92,0.02,0.88,0.95


[1] "Safety, Speed = [-1, 1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.28,0.05,0.17,0.39


[1] "Safety, Speed = [-1, -1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.14,0.03,0.07,0.2


# Accordance rates of Corrected Safety Prioritized Model

Change those decisions where one (1) unknown door is involved. The cue order remains to be Safety > Speed. However, an Unknown door is (always) better than an Unsafe one.

In [116]:
# Copy TTB Safety
dat$chosen_ttb_safety_corrected <- dat$chosen_ttb_safety

## Make changes to those decisions where an unknown door is involved
dat[dat$unknown_door_involved & dat$left_safety == 1 , 'chosen_ttb_safety_corrected'] <- 'left'
dat[dat$unknown_door_involved & dat$left_safety == -1 , 'chosen_ttb_safety_corrected'] <- 'right'

dat[dat$unknown_door_involved & dat$right_safety == 1 , 'chosen_ttb_safety_corrected'] <- 'right'
dat[dat$unknown_door_involved & dat$right_safety == -1 , 'chosen_ttb_safety_corrected'] <- 'left'

### Corrected Safety: Overall accordance rate

In [132]:
dat %>%
    filter( equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

dat %>%
    filter( equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

dat %>%
    filter( equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety_corrected )

accordance_rate_mean,se,lower_bound,upper_bound
0.73,0.01,0.7,0.75


accordance_rate_mean,se,lower_bound,upper_bound
0.66,0.01,0.64,0.69


accordance_rate_mean,se,lower_bound,upper_bound
0.89,0.01,0.86,0.92


### 4. Corrected Safety: Unknown door is involved

In [133]:
print('Safety')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

print('Corrected Safety')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety_corrected )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


[1] "Corrected Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.87,0.02,0.83,0.91


### 4.2 Corrected Safety: Unknown door is involved. Digging deeper

In [ ]:
print("Safety, Speed = [1, 1]")
safety_val = 1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [1, -1]")
safety_val = 1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, 1]")
safety_val = -1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, -1]")
safety_val = -1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )

# Response times

### Overall response time per block
It is calculated as an average of averages.

In [ ]:
response_times <- dat %>%
                    group_by(id, block) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block)

response_times

In [ ]:
p1 = bar_graph(response_times)

In [ ]:
#p1

## Hypotheses comparison

TTB_Safety vs TTB_Speed

In [102]:
bar_graph_2 <- function(plot_data, title) {
    plot_data %>%
        ggplot(aes(x=factor(x), y=y, group=group, fill=factor(group)))+
        geom_bar(stat="identity", position=position_dodge()) +
        geom_errorbar(aes(ymin=lower_bound, ymax=upper_bound),
                      position=position_dodge(.9),
                      width=0.2 ) +
        labs(fill = "Cues checked") + xlab("Block") + ylab("Response time (ms.)") + ggtitle(title)
}

In [103]:
rt_ttb_safety <- dat %>%
                    group_by(id, block, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_safety)

p1 <- bar_graph_2(rt_ttb_safety, "(a)")

In [104]:
rt_ttb_safety <- dat %>%
                    filter(unknown_door_involved == FALSE) %>% # Unknown doors filtered out
                    group_by(id, block, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_safety)

p12 <- bar_graph_2(rt_ttb_safety, "(c)")

In [105]:
rt_ttb_speed <- dat %>%
                    group_by(id, block, steps_ttb_speed) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_speed) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_speed)

p2 <- bar_graph_2(rt_ttb_speed, "(b)")

In [106]:
rt_ttb_speed <- dat %>%
                    filter(unknown_door_involved == FALSE) %>% # Unknown doors filtered out
                    group_by(id, block, steps_ttb_speed) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_speed) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_speed)

p22 <- bar_graph_2(rt_ttb_speed, "(d)")

In [107]:
svg("Images/response_times.svg", width=8, height=4)
multiplot(p1, p12, p2, p22, cols=2)
dev.off()

png 
  2

### Only known doors: response times

In [ ]:
response_times <- dat %>%
                    filter(unknown_door_involved == FALSE) %>%
                    group_by(id, block) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block)

response_times

In [ ]:
p2 = bar_graph(response_times)

### Individual response times

In [ ]:
response_times <- dat %>%
                    group_by(id) %>%
                    summarize(y = mean(rt),
                              lower_bound = t.test(rt, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(rt, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x=id)

head(response_times)

In [ ]:
response_times %>%
    summarize(mean_rt = mean(y),
              lower_bound = t.test(y, conf.level=0.95)$conf.int[1],
              upper_bound = t.test(y, conf.level=0.95)$conf.int[2])

#### Separated between those decisions that involved unknown doors and those that did not

In [ ]:
response_times <- dat %>%
                    filter(id==1 | id==2 | id==4) %>%
                    group_by(id, unknown_door_involved) %>%
                    summarize(y = mean(rt),
                              se = se(rt))  %>%
                    mutate(lower_bound = y - se/2, 
                           upper_bound = y + se/2, 
                           x=paste(id, unknown_door_involved))

head(response_times)

# Explore

In [ ]:
dat %>%
    select(-KEY_RESPONSE_TIME, -DISPLAY_TIME, -row_id, -unknown_door_involved) %>%
    filter(chosen_ttb_speed != chosen_ttb_safety) %>%
    head